formation energy per atom 의 결과물을 reproduce 하는 코드

MAE 와 R2 score 분석을 통해 모델 성능 확인

In [ ]:
# 할당받은 GPU 성능 확인
!nvidia-smi

In [ ]:
!pip install pymatgen==2020.11.11

In [ ]:
import pymatgen
dir(pymatgen)

# CGCNN 모델 학습

In [ ]:
import time
import sys
import os
import torch
import sklearn
import pandas as pd
import shutil

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import metrics

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
def check_time(init_time, fin_time):
    time_interval = fin_time - init_time
    hour = time_interval // 3600
    time_interval = time_interval - 3600 * hour
    min = time_interval // 60
    time_interval = time_interval - 60 * min
    sec = round(time_interval)

    return("{} hours {} mins {} secs".format(hour, min, sec))

In [ ]:
def data_plot(path = "/content/drive/MyDrive/packages/cgcnn/cgcnn-master/results/formation_energy/test_results.csv"):
    dt = pd.read_csv(path, header = None)
    predicted_data = list(dt.iloc[:,2])
    real_data = list(dt.iloc[:,1])

    test_number = len(predicted_data)

    RMSE = mean_squared_error(real_data, predicted_data, squared = False)
    MAE = mean_absolute_error(real_data, predicted_data)
    r2 = r2_score(real_data, predicted_data)
    
    fig, ax = plt.subplots()

    ax.hist2d(real_data, predicted_data, norm=LogNorm(), bins=(100,100), cmap='Blues', alpha=0.7)

    ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--')

    ax.set_xlabel('MP band gap (eV)')
    ax.set_ylabel('ML band gap (eV)')

    fig.set_size_inches(5, 5)
    fig.tight_layout()

    plt.show()

    print("\n"+"test number : {}, RMSE : {}, MAE : {}, R2 : {}".format(test_number, RMSE, MAE, r2))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_path = '/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy'
sys.path.insert(0, my_path)

In [ ]:
os.chdir('/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy')

In [ ]:
dir_list = os.listdir("/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy")

In [ ]:
len(dir_list)

# 학습 시작

In [ ]:
time_1 = time.time()

!python ../../../main.py ./ --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 --epochs 1000 --n-conv 4 --h-fea-len 32 --lr 0.02 --lr-milestones 800 --print-freq 120

time_2 = time.time()

In [ ]:
check_time(time_1, time_2)

In [ ]:
data_plot("/content/drive/MyDrive/packages/cgcnn/cgcnn-master/results/formation_energy/test_results.csv")